## Importing necessary libraries

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image, ImageOps
import io

import torch
import torchvision
from torchvision import transforms, datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tkinter as tk
from tkinter import ttk, colorchooser

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0') 
    print('GPU')
else:
    device = torch.device('cpu')
    print('CPU')

GPU


## Loading MNIST dataset for hand-drawn digits

In [3]:
train = datasets.MNIST('', train=True, download=True,transform=transforms.Compose([transforms.ToTensor()]))
train_set = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)

test = datasets.MNIST('', train=False, download=True,transform=transforms.Compose([transforms.ToTensor()]))
test_set = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)

c:\users\jovan dmitrovic\appdata\local\programs\python\python37\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Creating Fully Conected Model as classifier for Handwritten Digits

In [4]:
class FullyConectedModel(nn.Module):
    
    # constructor
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
        
    # feed forward function
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        output = F.log_softmax(self.fc4(x), dim=1)
        
        return output

## Training and testing

In [5]:
model = FullyConectedModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

# training model on training dataset
for epoch in range(num_epochs):
    for data in train_set: 
        X, y = data
        X, y = X.to(device), y.to(device)
        model.zero_grad()  
        output = model(X.view(-1,28*28)) 
        loss = F.nll_loss(output, y) # nll stands for negative log-likelihood
        loss.backward() 
        optimizer.step()
    print('Epoch ' + str(epoch+1) + ' -------> Loss: ' + str(loss.cpu().detach().numpy()))  
    
# evaluation of model on testing dataset
correct = 0
total = 0

with torch.no_grad():
    for data in test_set:
        X, y = data
        X, y = X.to(device), y.to(device)
        output = model(X.view(-1,28*28))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1

print('Accuracy on test data: ' + str(round(correct/total, 3)))
torch.save(model, 'models/mnist_fc.pt')
# model = torch.load('models/mnist_fc.pt')

Epoch 1 -------> Loss: 0.22930302
Epoch 2 -------> Loss: 0.26051372
Epoch 3 -------> Loss: 0.06764458
Epoch 4 -------> Loss: 0.050637588
Epoch 5 -------> Loss: 0.0095831165
Epoch 6 -------> Loss: 0.142099
Epoch 7 -------> Loss: 0.108616926
Epoch 8 -------> Loss: 0.07913689
Epoch 9 -------> Loss: 0.011538881
Epoch 10 -------> Loss: 0.078684844
Accuracy on test data: 0.974


## Creating Convolutional Model (VGG like) as classifier for Handwritten Digits

In [6]:
class ConvModel(nn.Module):
    
    # constructor
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.fc1 = nn.Linear(1024, 100)
        self.fc2 = nn.Linear(100, 10)

    # feed forward function
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = F.log_softmax(self.fc2(x), dim=1)
        
        return output
 

## Training and testing

In [7]:
model = ConvModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 15

# training model on training dataset
for epoch in range(num_epochs):
    for data in train_set: 
        X, y = data
        X, y = X.to(device), y.to(device)
        model.zero_grad()  
        output = model(X.view(-1,1,28,28)) 
        loss = F.nll_loss(output, y) # nll stands for negative log-likelihood
        loss.backward() 
        optimizer.step()
    print('Epoch ' + str(epoch+1) + ' -------> Loss: ' + str(loss.cpu().detach().numpy()))  
    
# evaluation of model on testing dataset
correct = 0
total = 0

with torch.no_grad():
    for data in test_set:
        X, y = data
        X, y = X.to(device), y.to(device)
        output = model(X.view(-1,1,28,28))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1

print('Accuracy on test data: ' + str(round(correct/total, 3)))
torch.save(model, 'models/mnist_cnn.pt')
# model = torch.load('models/mnist_cnn.pt')

Epoch 1 -------> Loss: 0.009229989
Epoch 2 -------> Loss: 0.12891479
Epoch 3 -------> Loss: 0.011686894
Epoch 4 -------> Loss: 0.074636534
Epoch 5 -------> Loss: 0.020816639
Epoch 6 -------> Loss: 0.01277717
Epoch 7 -------> Loss: 0.005591956
Epoch 8 -------> Loss: 0.018048646
Epoch 9 -------> Loss: 0.010655026
Epoch 10 -------> Loss: 0.00015884734
Epoch 11 -------> Loss: 0.0019440919
Epoch 12 -------> Loss: 0.0003182065
Epoch 13 -------> Loss: 1.69082e-05
Epoch 14 -------> Loss: 0.00022395699
Epoch 15 -------> Loss: 2.2910338e-06
Accuracy on test data: 0.991
